# WASDE report VS wheat price 

## Import necessary libraries

In [1]:
# Sources:
# https://www.usda.gov/oce/commodity-markets/wasde/historical-wasde-report-data
# https://www.investing.com/indices/bloomberg-commodity-historical-data
# https://www.marketwatch.com/investing/future/w.1/download-data?startDate=4/1/2010&endDate=8/31/2010
# https://www.imf.org/en/Home

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import re

In [2]:
# Legend:
# For exact commodity/commodity group
# P – Production
# D – Domestic
# BS – Beginning Stock
# E-Export
# ES – Ending Stock
# S -Supply

# For exact region
# P_All – All Production
# D_All – All Domestic
# BS_All – All Beginning Stock
# E_All – All Export
# ES_All – All Ending Stock
# S_All – All Supply

## Merge data into one CSV/PKL from CSV files
period: April 2010 - August 2022

In [3]:
 # Get the list of all files and directories
path = "C:/Users/Yuriy Podmogaev/Desktop/Agro/data"                             
list_wasde = os.listdir(path)

# Collect dataset from all files (period 2010.04 - 2022.07)
df1 = pd.read_csv('data/oce-wasde-report-data-2010-04-to-2015-12.csv')           
for i in range(1,len(list_wasde)):
    df2 = pd.read_csv(str(f'data/{list_wasde[i]}'))
    df1 = pd.concat([df1, df2])
    
# Save file in proper format.
df1.to_pickle('wasde-WCCRS-2010.04-to-2022.07.pkl')

C:\Users\Yuriy Podmogaev\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3194: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Regroup dataset

In [4]:
# Open to continue
wasde_10_22 = pd.read_pickle('wasde-WCCRS-2010.04-to-2022.07.pkl')
wasde_10_22['ReleaseDate'] = pd.to_datetime(wasde_10_22['ReleaseDate'], dayfirst =True)

#  Establish required reports/columns
Report_Title = ['World Wheat Supply and Use', 'World Coarse Grain Supply and Use', 'World Corn Supply and Use', 
                'World Rice Supply and Use  (Milled Basis)', 'World Soybean Supply and Use']
Columns = ['ReportDate', 'ReleaseDate', 'Region', 'Commodity', 'Attribute', 'MarketYear', 'ForecastMonth', 'Value']

#### Regroup set for world production
Wheat, Corn, Coars, SBean, Rice

In [5]:
atr = ['Production']
atr2 = 'P'
World_Agro =  wasde_10_22[(wasde_10_22['ReportTitle'].isin(Report_Title)) & (wasde_10_22['Region'] == 'World') 
                         & (wasde_10_22['Attribute'].isin(atr) )][Columns]

# Removing irrelevant dates(remaining latets market year(shift after 4th month))
World_Agro = World_Agro[((World_Agro['MarketYear'].map(lambda x: x.split('/')[1]) == World_Agro['ReportDate'].map(lambda x: x[-2:])) 
        & (World_Agro['ForecastMonth'] <= 4))
        | (World_Agro['MarketYear'].map(lambda x: x.split('/')[1]) == (World_Agro['ReportDate'].map(lambda x: str(int(x[-2:])+1))))]

# Extracting set for wheat
World_Wheat =  World_Agro[World_Agro['Commodity'] == 'Wheat']
World_Wheat.rename(columns = {'Value':f'{atr2}_Wheat'}, inplace = True)
World_Wheat = World_Wheat.drop(['Commodity'], axis=1)

# Extracting set for corn
World_Corn =  World_Agro[World_Agro['Commodity'] == 'Corn']
World_Corn.rename(columns = {'Value': f'{atr2}_Corn'}, inplace = True)
World_Corn = World_Corn.drop(['Commodity'], axis=1)

# Extracting set for Coarse Grain
World_Coarse =  World_Agro[World_Agro['Commodity'] == 'Coarse Grain']
World_Coarse.rename(columns = {'Value': f'{atr2}_Coarse'}, inplace = True)
World_Coarse = World_Coarse.drop(['Commodity'], axis=1)

# Extracting set for Rice
World_Rice =  World_Agro[World_Agro['Commodity'] == 'Rice']
World_Rice.rename(columns = {'Value': f'{atr2}_Rice'}, inplace = True)
World_Rice = World_Rice.drop(['Commodity'], axis=1)

# Extracting set for Oilseed, Soybean
World_Soybean =  World_Agro[World_Agro['Commodity'] == 'Oilseed, Soybean']
World_Soybean.rename(columns = {'Value': f'{atr2}_Soyb'}, inplace = True)
World_Soybean = World_Soybean.drop(['Commodity'], axis=1)

C:\Users\Yuriy Podmogaev\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [6]:
# Collecting all at one dataset.
World_Agro = pd.merge(World_Wheat, World_Corn[['ReportDate', f'{atr2}_Corn']])
World_Agro = pd.merge(World_Agro, World_Coarse[['ReportDate', f'{atr2}_Coarse']])
World_Agro = pd.merge(World_Agro, World_Rice[['ReportDate', f'{atr2}_Rice']])
World_Agro = pd.merge(World_Agro, World_Soybean[['ReportDate', f'{atr2}_Soyb']])

In [7]:
# Save file in proper format.
World_Agro.to_pickle('World_Agro_WASDE.pkl')

In [8]:
World_Agro_WASDE = pd.read_pickle('World_Agro_WASDE.pkl')
World_Agro_WASDE.head(2)

,ReportDate,ReleaseDate,Region,Attribute,MarketYear,ForecastMonth,P_Wheat,P_Corn,P_Coarse,P_Rice,P_Soyb
0,April 2010,2010-04-09,World,Production,2009/10,4,678.42,805.68,1102.84,440.82,257.46
1,May 2010,2010-05-11,World,Production,2010/11,5,672.18,835.03,1129.77,459.74,250.13


#### Regroup set for world beginning stocks
Wheat, Corn, Coars, SBean, Rice

In [9]:
atr = ['Beginning Stocks', 'Beginning stocks']
atr2 = 'BS'
World_Agro =  wasde_10_22[(wasde_10_22['ReportTitle'].isin(Report_Title)) & (wasde_10_22['Region'] == 'World') 
                         & (wasde_10_22['Attribute'].isin(atr) )][Columns]

# Removing irrelevant dates(remaining latets market year(shift after 4th month))
World_Agro = World_Agro[((World_Agro['MarketYear'].map(lambda x: x.split('/')[1]) == World_Agro['ReportDate'].map(lambda x: x[-2:])) 
        & (World_Agro['ForecastMonth'] <= 4))
        | (World_Agro['MarketYear'].map(lambda x: x.split('/')[1]) == (World_Agro['ReportDate'].map(lambda x: str(int(x[-2:])+1))))]
World_Agro.head(2)

# Extracting set for wheat
World_Wheat =  World_Agro[World_Agro['Commodity'] == 'Wheat']
World_Wheat.rename(columns = {'Value':f'{atr2}_Wheat'}, inplace = True)
World_Wheat = World_Wheat.drop(['Commodity'], axis=1)

# Extracting set for corn
World_Corn =  World_Agro[World_Agro['Commodity'] == 'Corn']
World_Corn.rename(columns = {'Value': f'{atr2}_Corn'}, inplace = True)
World_Corn = World_Corn.drop(['Commodity'], axis=1)

# Extracting set for Coarse Grain
World_Coarse =  World_Agro[World_Agro['Commodity'] == 'Coarse Grain']
World_Coarse.rename(columns = {'Value': f'{atr2}_Coarse'}, inplace = True)
World_Coarse = World_Coarse.drop(['Commodity'], axis=1)

# Extracting set for Rice
World_Rice =  World_Agro[World_Agro['Commodity'] == 'Rice']
World_Rice.rename(columns = {'Value': f'{atr2}_Rice'}, inplace = True)
World_Rice = World_Rice.drop(['Commodity'], axis=1)

# Extracting set for Oilseed, Soybean
World_Soybean =  World_Agro[World_Agro['Commodity'] == 'Oilseed, Soybean']
World_Soybean.rename(columns = {'Value': f'{atr2}_Soyb'}, inplace = True)
World_Soybean = World_Soybean.drop(['Commodity'], axis=1)

C:\Users\Yuriy Podmogaev\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [10]:
# Collecting all at one dataset.
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Wheat[['ReportDate', f'{atr2}_Wheat']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Corn[['ReportDate', f'{atr2}_Corn']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Coarse[['ReportDate', f'{atr2}_Coarse']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Rice[['ReportDate', f'{atr2}_Rice']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Soybean[['ReportDate', f'{atr2}_Soyb']])

In [11]:
# Save file in proper format.
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')

In [12]:
World_Agro_WASDE = pd.read_pickle('World_Agro_WASDE.pkl')
World_Agro_WASDE.head(2)

,ReportDate,ReleaseDate,Region,Attribute,MarketYear,ForecastMonth,P_Wheat,P_Corn,P_Coarse,P_Rice,P_Soyb,BS_Wheat,BS_Corn,BS_Coarse,BS_Rice,BS_Soyb
0,April 2010,2010-04-09,World,Production,2009/10,4,678.42,805.68,1102.84,440.82,257.46,165.23,147.50,192.98,90.90,42.82
1,May 2010,2010-05-11,World,Production,2010/11,5,672.18,835.03,1129.77,459.74,250.13,193.37,147.04,196.70,90.32,63.76


#### Regroup set for world domestic
Wheat, Corn, Coars, SBean, Rice

In [13]:
atr = ['Total  Domestic', 'Total Domestic', 'Domestic Total', 'Domestic, Total', 'Domestic, total']
atr2 = 'D'
World_Agro =  wasde_10_22[(wasde_10_22['ReportTitle'].isin(Report_Title)) & (wasde_10_22['Region'] == 'World') 
                         & (wasde_10_22['Attribute'].isin(atr) )][Columns]

# Removing irrelevant dates(remaining latets market year(shift after 4th month))
World_Agro = World_Agro[((World_Agro['MarketYear'].map(lambda x: x.split('/')[1]) == World_Agro['ReportDate'].map(lambda x: x[-2:])) 
        & (World_Agro['ForecastMonth'] <= 4))
        | (World_Agro['MarketYear'].map(lambda x: x.split('/')[1]) == (World_Agro['ReportDate'].map(lambda x: str(int(x[-2:])+1))))]
World_Agro.head(2)

# Extracting set for wheat
World_Wheat =  World_Agro[World_Agro['Commodity'] == 'Wheat']
World_Wheat.rename(columns = {'Value':f'{atr2}_Wheat'}, inplace = True)
World_Wheat = World_Wheat.drop(['Commodity'], axis=1)

# Extracting set for corn
World_Corn =  World_Agro[World_Agro['Commodity'] == 'Corn']
World_Corn.rename(columns = {'Value': f'{atr2}_Corn'}, inplace = True)
World_Corn = World_Corn.drop(['Commodity'], axis=1)

# Extracting set for Coarse Grain
World_Coarse =  World_Agro[World_Agro['Commodity'] == 'Coarse Grain']
World_Coarse.rename(columns = {'Value': f'{atr2}_Coarse'}, inplace = True)
World_Coarse = World_Coarse.drop(['Commodity'], axis=1)

# Extracting set for Rice
World_Rice =  World_Agro[World_Agro['Commodity'] == 'Rice']
World_Rice.rename(columns = {'Value': f'{atr2}_Rice'}, inplace = True)
World_Rice = World_Rice.drop(['Commodity'], axis=1)

# Extracting set for Oilseed, Soybean
World_Soybean =  World_Agro[World_Agro['Commodity'] == 'Oilseed, Soybean']
World_Soybean.rename(columns = {'Value': f'{atr2}_Soyb'}, inplace = True)
World_Soybean = World_Soybean.drop(['Commodity'], axis=1)

C:\Users\Yuriy Podmogaev\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [14]:
# Collecting all at one dataset.
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Wheat[['ReportDate', f'{atr2}_Wheat']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Corn[['ReportDate', f'{atr2}_Corn']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Coarse[['ReportDate', f'{atr2}_Coarse']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Rice[['ReportDate', f'{atr2}_Rice']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Soybean[['ReportDate', f'{atr2}_Soyb']])

In [15]:
# Save file in proper format.
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')

In [16]:
World_Agro_WASDE = pd.read_pickle('World_Agro_WASDE.pkl')
World_Agro_WASDE.head(2)

,ReportDate,ReleaseDate,Region,Attribute,MarketYear,ForecastMonth,P_Wheat,P_Corn,P_Coarse,P_Rice,...,BS_Wheat,BS_Corn,BS_Coarse,BS_Rice,BS_Soyb,D_Wheat,D_Corn,D_Coarse,D_Rice,D_Soyb
0,April 2010,2010-04-09,World,Production,2009/10,4,678.42,805.68,1102.84,440.82,...,165.23,147.50,192.98,90.90,42.82,647.83,808.98,1105.76,441.51,235.69
1,May 2010,2010-05-11,World,Production,2010/11,5,672.18,835.03,1129.77,459.74,...,193.37,147.04,196.70,90.32,63.76,667.46,827.87,1124.96,453.44,246.35


#### Regroup set for world export
Wheat, Corn, Coars, SBean, Rice

In [17]:
atr = ['Exports']
atr2 = 'E'
World_Agro =  wasde_10_22[(wasde_10_22['ReportTitle'].isin(Report_Title)) & (wasde_10_22['Region'] == 'World') 
                         & (wasde_10_22['Attribute'].isin(atr) )][Columns]

# Removing irrelevant dates(remaining latets market year(shift after 4th month))
World_Agro = World_Agro[((World_Agro['MarketYear'].map(lambda x: x.split('/')[1]) == World_Agro['ReportDate'].map(lambda x: x[-2:])) 
        & (World_Agro['ForecastMonth'] <= 4))
        | (World_Agro['MarketYear'].map(lambda x: x.split('/')[1]) == (World_Agro['ReportDate'].map(lambda x: str(int(x[-2:])+1))))]
World_Agro.head(2)

# Extracting set for wheat
World_Wheat =  World_Agro[World_Agro['Commodity'] == 'Wheat']
World_Wheat.rename(columns = {'Value':f'{atr2}_Wheat'}, inplace = True)
World_Wheat = World_Wheat.drop(['Commodity'], axis=1)

# Extracting set for corn
World_Corn =  World_Agro[World_Agro['Commodity'] == 'Corn']
World_Corn.rename(columns = {'Value': f'{atr2}_Corn'}, inplace = True)
World_Corn = World_Corn.drop(['Commodity'], axis=1)

# Extracting set for Coarse Grain
World_Coarse =  World_Agro[World_Agro['Commodity'] == 'Coarse Grain']
World_Coarse.rename(columns = {'Value': f'{atr2}_Coarse'}, inplace = True)
World_Coarse = World_Coarse.drop(['Commodity'], axis=1)

# Extracting set for Rice
World_Rice =  World_Agro[World_Agro['Commodity'] == 'Rice']
World_Rice.rename(columns = {'Value': f'{atr2}_Rice'}, inplace = True)
World_Rice = World_Rice.drop(['Commodity'], axis=1)

# Extracting set for Oilseed, Soybean
World_Soybean =  World_Agro[World_Agro['Commodity'] == 'Oilseed, Soybean']
World_Soybean.rename(columns = {'Value': f'{atr2}_Soyb'}, inplace = True)
World_Soybean = World_Soybean.drop(['Commodity'], axis=1)

C:\Users\Yuriy Podmogaev\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [18]:
# Collecting all at one dataset.
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Wheat[['ReportDate', f'{atr2}_Wheat']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Corn[['ReportDate', f'{atr2}_Corn']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Coarse[['ReportDate', f'{atr2}_Coarse']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Rice[['ReportDate', f'{atr2}_Rice']])
World_Agro_WASDE = pd.merge(World_Agro_WASDE, World_Soybean[['ReportDate', f'{atr2}_Soyb']])

In [19]:
# Save file in proper format.
World_Agro_WASDE = World_Agro_WASDE.drop('Attribute', axis=1)
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')

In [20]:
World_Agro_WASDE = pd.read_pickle('World_Agro_WASDE.pkl')
World_Agro_WASDE.head(2)

,ReportDate,ReleaseDate,Region,MarketYear,ForecastMonth,P_Wheat,P_Corn,P_Coarse,P_Rice,P_Soyb,...,D_Wheat,D_Corn,D_Coarse,D_Rice,D_Soyb,E_Wheat,E_Corn,E_Coarse,E_Rice,E_Soyb
0,April 2010,2010-04-09,World,2009/10,4,678.42,805.68,1102.84,440.82,257.46,...,647.83,808.98,1105.76,441.51,235.69,125.89,86.01,110.97,30.04,82.41
1,May 2010,2010-05-11,World,2010/11,5,672.18,835.03,1129.77,459.74,250.13,...,667.46,827.87,1124.96,453.44,246.35,129.18,88.53,112.83,31.42,87.92


## Merge data into one CSV/PKL from TXT files
period: Jan 2000 - Mar 2010

#### Collect world wheat

In [21]:
# Set column names for extraxtion
head = ['ReleaseDate','Month', ':','BS_Wheat', 'P_Wheat', 'Imports', 'Feed', 'D_Wheat', 'E_Wheat', 'Ending stocks']

# Get the list for data and directories
old_wasde = []
path = "C:/Users/Yuriy Podmogaev/Desktop/Agro/dataold"                             
list_wasde_old = os.listdir(path)

# Grab data
for j in list_wasde_old:
    fn = j
    with open(f'dataold/{fn}') as f:
        text = f.readlines()
    flag = False
    for i in range(len(text)):
        match = re.findall(r"World Wheat Supply and Use", text[i])
        match2 = re.findall(r"Cont", text[i])
        if len(match) > 0 and len(match2) > 0:
            flag = True
        math3 = re.findall(r"United", text[i])
        if len(math3) > 0 and flag:
            old_wasde.append([str(fn[-14:-4])]+text[i-1].split())
            flag = False

# Combine and smooth dataset
df2 = pd.DataFrame(old_wasde, columns = head)
df2['ReleaseDate'] = pd.to_datetime(df2['ReleaseDate'], format='%m-%d-%Y')
df2['Year_Month'] = pd.to_datetime(df2['ReleaseDate']).dt.to_period('M')
df2 = df2.sort_values(by='ReleaseDate')
df2['Month'] = df2['Year_Month'].map(lambda x: int(str(x)[-2:]))
df2.rename(columns={'Month':'ForecastMonth'}, inplace=True)
world_w = df2[['ReleaseDate','ForecastMonth', 'BS_Wheat', 'P_Wheat', 'D_Wheat', 'E_Wheat', 'Year_Month']]
world_w.head(2)

,ReleaseDate,ForecastMonth,BS_Wheat,P_Wheat,D_Wheat,E_Wheat,Year_Month
4,2000-01-12,1,136.01,584.45,591.08,126.19,2000-01
20,2000-02-11,2,135.57,584.80,593.13,125.26,2000-02


#### Collect Coarse grains

In [22]:
# Set column names for extraxtion
head = ['ReleaseDate','ForecastMonth', ':','BS_Coarse', 'P_Coarse', 'Imports', 'Feed', 'D_Coarse', 'E_Coarse', 'Ending stocks']

# Get the list for data and directories
old_wasde = []
path = "C:/Users/Yuriy Podmogaev/Desktop/Agro/dataold"                             
list_wasde_old = os.listdir(path)

# Grab data
for j in list_wasde_old:
    fn = j
    with open(f'dataold/{fn}') as f:
        text = f.readlines()
    flag = False
    for i in range(len(text)):
        match = re.findall(r"World Coarse Grain Supply and Use", text[i])
        match2 = re.findall(r"Cont", text[i])
        if len(match) > 0 and len(match2) > 0:
            flag = True
        math3 = re.findall(r"United", text[i])
        if len(math3) > 0 and flag:
            old_wasde.append([str(fn[-14:-4])]+text[i-1].split())
            flag = False

# Combine and smooth dataset
df2 = pd.DataFrame(old_wasde, columns = head)
df2['ReleaseDate'] = pd.to_datetime(df2['ReleaseDate'], format='%m-%d-%Y')
df2['Year_Month'] = pd.to_datetime(df2['ReleaseDate']).dt.to_period('M')
df2 = df2.sort_values(by='ReleaseDate')
world_coarse = df2[['BS_Coarse', 'P_Coarse', 'D_Coarse', 'E_Coarse', 'Year_Month']]
world_coarse.head(2)

,BS_Coarse,P_Coarse,D_Coarse,E_Coarse,Year_Month
4,153.75,873.71,878.45,110.79,2000-01
20,155.81,873.14,880.77,113.58,2000-02


#### Collect corn

In [23]:
# Set column names for extraxtion
head = ['ReleaseDate','Month', ':','BS_Corn', 'P_Corn', 'Imports', 'Feed', 'D_Corn', 'E_Corn', 'Ending stocks']

# Get the list for data and directories
old_wasde = []
path = "C:/Users/Yuriy Podmogaev/Desktop/Agro/dataold"                             
list_wasde_old = os.listdir(path)

# Grab data
for j in list_wasde_old:
    fn = j
    with open(f'dataold/{fn}') as f:
        text = f.readlines()
    flag = False
    for i in range(len(text)): # Использовать итератор файла
        match = re.findall(r"World Corn Supply and Use", text[i])
        match2 = re.findall(r"Cont", text[i])
        if len(match) > 0 and len(match2) > 0:
            flag = True
        math3 = re.findall(r"United", text[i])
        if len(math3) > 0 and flag:
            old_wasde.append([str(fn[-14:-4])]+text[i-1].split())
            flag = False

# Combine and smooth dataset
df2 = pd.DataFrame(old_wasde, columns = head)
df2['ReleaseDate'] = pd.to_datetime(df2['ReleaseDate'], format='%m-%d-%Y')
df2['Year_Month'] = pd.to_datetime(df2['ReleaseDate']).dt.to_period('M')
df2 = df2.sort_values(by='ReleaseDate')
world_corn = df2[['BS_Corn', 'P_Corn', 'D_Corn', 'E_Corn', 'Year_Month']]
world_corn.head(2)

,BS_Corn,P_Corn,D_Corn,E_Corn,Year_Month
4,108.44,597.99,596.90,78.12,2000-01
20,109.32,599.93,600.87,80.34,2000-02


#### Collect Rice

In [24]:
# Set column names for extraxtion
head = ['ReleaseDate','Month', ':','BS_Rice', 'P_Rice', 'Imports', 'D_Rice', 'E_Rice', 'Ending stocks']

# Get the list for data and directories
old_wasde = []
path = "C:/Users/Yuriy Podmogaev/Desktop/Agro/dataold"                             
list_wasde_old = os.listdir(path)

# Grab data
for j in list_wasde_old:
    fn = j
    with open(f'dataold/{fn}') as f:
        text = f.readlines()
    flag = False
    cnt = 0
    for i in range(len(text)): # Использовать итератор файла
        match = re.findall(r"World Rice Supply and Use", text[i])
        if len(match) > 0:
            flag = True
        math3 = re.findall(r"United", text[i])
        if len(math3) > 0 and flag:
            cnt +=1
        if cnt == 3 and flag:
            old_wasde.append([str(fn[-14:-4])]+text[i-1].split())
            flag = False
for i in old_wasde:
    if len(i) == 11:
        del i[1]
        del i[1]

# Combine and smooth dataset
df2 = pd.DataFrame(old_wasde, columns = head)
df2['ReleaseDate'] = pd.to_datetime(df2['ReleaseDate'], format='%m-%d-%Y')
df2['Year_Month'] = pd.to_datetime(df2['ReleaseDate']).dt.to_period('M')
df2 = df2.sort_values(by='ReleaseDate')
world_rice = df2[['BS_Rice', 'P_Rice', 'D_Rice', 'E_Rice', 'Year_Month']]
world_rice.head(2)

,BS_Rice,P_Rice,D_Rice,E_Rice,Year_Month
4,57.13,396.51,394.82,23.98,2000-01
20,58.67,397.42,396.68,23.60,2000-02


#### Collect Soya bean

In [25]:
# Set column names for extraxtion
head = ['ReleaseDate','Month', ':','BS_Soyb', 'P_Soyb', 'Imports', 'Crush', 'D_Soyb', 'E_Soyb', 'Ending stocks']

# Get the list for data and directories
old_wasde = []
path = "C:/Users/Yuriy Podmogaev/Desktop/Agro/dataold"                             
list_wasde_old = os.listdir(path)

# Grab data
for j in list_wasde_old:
    fn = j
    with open(f'dataold/{fn}') as f:
        text = f.readlines()
    flag = False
    cnt = 0
    for i in range(len(text)): # Использовать итератор файла
        match = re.findall(r"World Soybean Supply and Use", text[i])
        if len(match) > 0:
            flag = True
        math3 = re.findall(r"United", text[i])
        if len(math3) > 0 and flag:
            cnt +=1
        if cnt == 3 and flag:
            old_wasde.append([str(fn[-14:-4])]+text[i-1].split())
            flag = False
for i in old_wasde:
    if len(i) == 9:
        print(i)

df2 = pd.DataFrame(old_wasde, columns = head)
df2['ReleaseDate'] = pd.to_datetime(df2['ReleaseDate'], format='%m-%d-%Y')
df2['Year_Month'] = pd.to_datetime(df2['ReleaseDate']).dt.to_period('M')
df2 = df2.sort_values(by='ReleaseDate')
world_soyb = df2[['BS_Soyb', 'P_Soyb', 'D_Soyb', 'E_Soyb', 'Year_Month']]
world_soyb.head(2)

,BS_Soyb,P_Soyb,D_Soyb,E_Soyb,Year_Month
4,24.02,153.75,155.94,41.12,2000-01
20,24.01,153.25,156.93,41.28,2000-02


#### Merge set

In [26]:
#Merge in one file
World_Agro_WASDE_00_10 = pd.merge(world_w, world_coarse,  on=('Year_Month'))
World_Agro_WASDE_00_10 = pd.merge(World_Agro_WASDE_00_10, world_corn,  on=('Year_Month'))
World_Agro_WASDE_00_10 = pd.merge(World_Agro_WASDE_00_10, world_rice,  on=('Year_Month'))
World_Agro_WASDE_00_10 = pd.merge(World_Agro_WASDE_00_10, world_soyb,  on=('Year_Month'))
World_Agro_WASDE_00_10 = World_Agro_WASDE_00_10.drop_duplicates(keep='first')
World_Agro_WASDE_00_10 = World_Agro_WASDE_00_10.reset_index(drop=True)

# Set proper type
co = ['BS_Wheat', 'P_Wheat', 'D_Wheat', 'E_Wheat', 'BS_Coarse', 'P_Coarse', 'D_Coarse', 'E_Coarse',
      'BS_Corn', 'P_Corn', 'D_Corn', 'E_Corn', 'BS_Rice', 'P_Rice', 'D_Rice', 'E_Rice', 'BS_Soyb', 
      'P_Soyb', 'D_Soyb', 'E_Soyb']
for i in co:
    World_Agro_WASDE_00_10[i] = World_Agro_WASDE_00_10[i].astype(float)
World_Agro_WASDE_00_10['Region'] = 'World'
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')
World_Agro_WASDE_00_10.head(2)

,ReleaseDate,ForecastMonth,BS_Wheat,P_Wheat,D_Wheat,E_Wheat,Year_Month,BS_Coarse,P_Coarse,D_Coarse,...,E_Corn,BS_Rice,P_Rice,D_Rice,E_Rice,BS_Soyb,P_Soyb,D_Soyb,E_Soyb,Region
0,2000-01-12,1,136.01,584.45,591.08,126.19,2000-01,153.75,873.71,878.45,...,78.12,57.13,396.51,394.82,23.98,24.02,153.75,155.94,41.12,World
1,2000-02-11,2,135.57,584.80,593.13,125.26,2000-02,155.81,873.14,880.77,...,80.34,58.67,397.42,396.68,23.60,24.01,153.25,156.93,41.28,World


## Merge two sets at one
period: Jan 2000 - Aug 2022

In [27]:
# Prepar main set for combination
World_Agro_WASDE['Year_Month'] = pd.to_datetime(World_Agro_WASDE['ReportDate']).dt.to_period('M')

# Merge and save
World_Agro_WASDE = pd.concat([World_Agro_WASDE_00_10, World_Agro_WASDE])

In [28]:
World_Agro_WASDE = World_Agro_WASDE.reset_index(drop=True)

In [29]:
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')

## Add bloomberg commodity index

In [30]:
# Download bloomberg commodity index, calculation average price
BCOM = pd.read_excel('BCOM/BCOM1.xlsx')
BCOM['Date'] = pd.to_datetime(BCOM['Date'], format='%m%d%Y')
BCOM['Ave_BCOM'] = (BCOM['Price'] + BCOM['Open'] + BCOM['High'] + BCOM['Low'])/4

# Preparing BCOM set for combination
BCOM['Year_Month'] = pd.to_datetime(BCOM['Date']).dt.to_period('M')

In [31]:
# Combiing sets 
World_Agro_WASDE = pd.merge(World_Agro_WASDE, BCOM[['Year_Month', 'Ave_BCOM']], on=('Year_Month'), how ='right')

In [32]:
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')
World_Agro_WASDE.head()

,ReleaseDate,ForecastMonth,BS_Wheat,P_Wheat,D_Wheat,E_Wheat,Year_Month,BS_Coarse,P_Coarse,D_Coarse,...,D_Rice,E_Rice,BS_Soyb,P_Soyb,D_Soyb,E_Soyb,Region,ReportDate,MarketYear,Ave_BCOM
0,2022-08-12,8.0,276.35,779.60,788.60,208.65,2022-08,338.30,1469.49,1476.15,...,518.74,54.67,89.73,392.79,378.25,169.08,World,August 2022,2022/23,121.032900
1,2022-07-12,7.0,280.10,771.64,784.22,205.47,2022-07,338.36,1477.20,1477.65,...,518.63,54.61,88.73,391.40,377.75,168.89,World,July 2022,2022/23,118.037850
2,2022-06-10,6.0,279.40,773.43,785.99,204.59,2022-06,336.30,1479.19,1479.80,...,519.22,54.20,86.15,395.37,377.86,170.31,World,June 2022,2022/23,125.993725
3,2022-05-12,5.0,279.72,774.83,787.52,204.89,2022-05,334.42,1475.87,1479.95,...,518.44,54.20,85.24,394.69,377.44,170.01,World,May 2022,2022/23,129.821625
4,2022-04-08,4.0,290.67,778.83,791.08,200.10,2022-04,321.35,1501.55,1492.66,...,511.19,52.46,103.11,350.72,361.88,155.29,World,April 2022,2021/22,128.242725


## Add SRWheat (CBOT, front month price)

In [33]:
# Download SRW price, calculation average price
SRW = pd.read_excel('SRW/SRW.xlsx')
SRW['Year_Month'] = pd.to_datetime(SRW['Date']).dt.to_period('M')
SRW['Ave_SRW'] = (SRW['Open'] + SRW['High'] + SRW['Low'] + SRW['Price'])/4

# Download inflation set with accumulated inflation coificient('AIC')
INFL = pd.read_excel('inflation/inflation_world.xlsx')
INFL['Year_Month'] = pd.to_datetime(INFL['Date']).dt.to_period('M')
INFL.head()

# Combiing price and inflation set
SRW = pd.merge(INFL[['AIC', 'Year_Month']], SRW[['Year_Month', 'Ave_SRW']], on=('Year_Month'), how ='right')

# Fill in inflation and cleaning inflation off price (contrary to NPV calculation)
SRW['AIC'] = SRW['AIC'].interpolate()
SRW['Ave_SRW'] = SRW['AIC']*SRW['Ave_SRW']
SRW.head()

# Combiing sets 
World_Agro_WASDE = pd.merge(World_Agro_WASDE, SRW[['Year_Month', 'Ave_SRW']], on=('Year_Month'), how ='right')

# Conver ¢/bush to USD/tonn
def c_bu_to_usd_t(price):
    return price * 0.367437

World_Agro_WASDE['Ave_SRW'] = World_Agro_WASDE['Ave_SRW'].apply(c_bu_to_usd_t)

In [34]:
# Save changes
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')
World_Agro_WASDE.head(2)

,ReleaseDate,ForecastMonth,BS_Wheat,P_Wheat,D_Wheat,E_Wheat,Year_Month,BS_Coarse,P_Coarse,D_Coarse,...,E_Rice,BS_Soyb,P_Soyb,D_Soyb,E_Soyb,Region,ReportDate,MarketYear,Ave_BCOM,Ave_SRW
0,2022-08-12,8.0,276.35,779.60,788.60,208.65,2022-08,338.30,1469.49,1476.15,...,54.67,89.73,392.79,378.25,169.08,World,August 2022,2022/23,121.03290,291.607189
1,2022-07-12,7.0,280.10,771.64,784.22,205.47,2022-07,338.36,1477.20,1477.65,...,54.61,88.73,391.40,377.75,168.89,World,July 2022,2022/23,118.03785,307.552121


## Fill in Nan
Filling in with mean meaning:

In [35]:
col = ['P_Wheat', 'P_Corn', 'P_Coarse', 'P_Rice', 'P_Soyb', 'BS_Wheat', 'BS_Corn', 'BS_Coarse', 'BS_Rice', 'BS_Soyb', 'D_Wheat', 
       'D_Corn', 'D_Coarse', 'D_Rice', 'D_Soyb', 'E_Wheat', 'E_Corn', 'E_Coarse', 'E_Rice', 'E_Soyb', 'Ave_BCOM', 'Ave_SRW']
for i in col:
    World_Agro_WASDE[i].replace(to_replace=0, value=np.nan, inplace=True)
    World_Agro_WASDE[i].interpolate(inplace=True)

In [36]:
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')

## Fill in missing and general values
Ending stocks for all and combined walues with all commodities

In [37]:
# Combine ending stocs values
World_Agro_WASDE['ES_Wheat'] = World_Agro_WASDE['BS_Wheat'] + World_Agro_WASDE['P_Wheat'] - World_Agro_WASDE['D_Wheat']
World_Agro_WASDE['ES_Corn'] = World_Agro_WASDE['BS_Corn'] + World_Agro_WASDE['P_Corn'] - World_Agro_WASDE['D_Corn']
World_Agro_WASDE['ES_Coarse'] = World_Agro_WASDE['BS_Coarse'] + World_Agro_WASDE['P_Coarse'] - World_Agro_WASDE['D_Coarse']
World_Agro_WASDE['ES_Soyb'] = World_Agro_WASDE['BS_Soyb'] + World_Agro_WASDE['P_Soyb'] - World_Agro_WASDE['D_Soyb']
World_Agro_WASDE['ES_Rice'] = World_Agro_WASDE['BS_Rice'] + World_Agro_WASDE['P_Rice'] - World_Agro_WASDE['D_Rice']

# Combine overal values
World_Agro_WASDE['P_All'] = World_Agro_WASDE['P_Wheat'] + World_Agro_WASDE['P_Coarse'] + World_Agro_WASDE['P_Rice'] + World_Agro_WASDE['P_Soyb']
World_Agro_WASDE['BS_All'] = World_Agro_WASDE['BS_Wheat'] + World_Agro_WASDE['BS_Coarse'] + World_Agro_WASDE['BS_Rice'] + World_Agro_WASDE['BS_Soyb']
World_Agro_WASDE['D_All'] = World_Agro_WASDE['D_Wheat'] + World_Agro_WASDE['D_Coarse'] + World_Agro_WASDE['D_Rice'] + World_Agro_WASDE['D_Soyb']
World_Agro_WASDE['E_All'] = World_Agro_WASDE['E_Wheat'] + World_Agro_WASDE['E_Coarse'] + World_Agro_WASDE['E_Rice'] + World_Agro_WASDE['E_Soyb']
World_Agro_WASDE['ES_All'] = World_Agro_WASDE['BS_All'] + World_Agro_WASDE['P_All'] - World_Agro_WASDE['D_All']
World_Agro_WASDE['S_All'] = World_Agro_WASDE['P_All'] - World_Agro_WASDE['BS_All']


In [38]:
World_Agro_WASDE.columns

Index(['ReleaseDate', 'ForecastMonth', 'BS_Wheat', 'P_Wheat', 'D_Wheat',
       'E_Wheat', 'Year_Month', 'BS_Coarse', 'P_Coarse', 'D_Coarse',
       'E_Coarse', 'BS_Corn', 'P_Corn', 'D_Corn', 'E_Corn', 'BS_Rice',
       'P_Rice', 'D_Rice', 'E_Rice', 'BS_Soyb', 'P_Soyb', 'D_Soyb', 'E_Soyb',
       'Region', 'ReportDate', 'MarketYear', 'Ave_BCOM', 'Ave_SRW', 'ES_Wheat',
       'ES_Corn', 'ES_Coarse', 'ES_Soyb', 'ES_Rice', 'P_All', 'BS_All',
       'D_All', 'E_All', 'ES_All', 'S_All'],
      dtype='object')

In [39]:
columns = ['BS_Wheat', 'P_Wheat', 'D_Wheat', 'E_Wheat', 'BS_Coarse', 'P_Coarse', 'D_Coarse',
       'E_Coarse', 'BS_Corn', 'P_Corn', 'D_Corn', 'E_Corn', 'BS_Rice', 'P_Rice', 'D_Rice', 'E_Rice', 
       'BS_Soyb', 'P_Soyb', 'D_Soyb', 'E_Soyb', 'P_All', 'S_All', 'BS_All', 'D_All', 'E_All', 'ES_Wheat', 
       'ES_Corn', 'ES_Coarse', 'ES_Soyb', 'ES_Rice', 'ES_All']

In [40]:
for i in columns:
    World_Agro_WASDE.columns = World_Agro_WASDE.columns.str.replace(i, f'{"World"}_{i}')

In [41]:
World_Agro_WASDE.columns

Index(['ReleaseDate', 'ForecastMonth', 'World_BS_Wheat', 'World_P_Wheat',
       'World_D_Wheat', 'World_E_Wheat', 'Year_Month', 'World_BS_Coarse',
       'World_P_Coarse', 'World_D_Coarse', 'World_E_Coarse', 'World_BS_Corn',
       'World_P_Corn', 'World_D_Corn', 'World_E_Corn', 'World_BS_Rice',
       'World_P_Rice', 'World_D_Rice', 'World_E_Rice', 'World_BS_Soyb',
       'World_P_Soyb', 'World_D_Soyb', 'World_E_Soyb', 'Region', 'ReportDate',
       'MarketYear', 'Ave_BCOM', 'Ave_SRW', 'World_ES_Wheat', 'World_ES_Corn',
       'World_ES_Coarse', 'World_ES_Soyb', 'World_ES_Rice', 'World_P_All',
       'BWorld_S_All', 'World_D_All', 'World_E_All', 'EWorld_S_All',
       'World_S_All'],
      dtype='object')

In [42]:
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')

In [43]:
World_Agro_WASDE = World_Agro_WASDE.rename(columns={'BWorld_S_All': 'World_BS_All', 'EWorld_S_All': 'World_ES_All'})

In [44]:
World_Agro_WASDE.columns

Index(['ReleaseDate', 'ForecastMonth', 'World_BS_Wheat', 'World_P_Wheat',
       'World_D_Wheat', 'World_E_Wheat', 'Year_Month', 'World_BS_Coarse',
       'World_P_Coarse', 'World_D_Coarse', 'World_E_Coarse', 'World_BS_Corn',
       'World_P_Corn', 'World_D_Corn', 'World_E_Corn', 'World_BS_Rice',
       'World_P_Rice', 'World_D_Rice', 'World_E_Rice', 'World_BS_Soyb',
       'World_P_Soyb', 'World_D_Soyb', 'World_E_Soyb', 'Region', 'ReportDate',
       'MarketYear', 'Ave_BCOM', 'Ave_SRW', 'World_ES_Wheat', 'World_ES_Corn',
       'World_ES_Coarse', 'World_ES_Soyb', 'World_ES_Rice', 'World_P_All',
       'World_BS_All', 'World_D_All', 'World_E_All', 'World_ES_All',
       'World_S_All'],
      dtype='object')

In [45]:
World_Agro_WASDE.to_pickle('World_Agro_WASDE.pkl')

In [46]:
World_Agro_WASDE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 0 to 271
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ReleaseDate      269 non-null    datetime64[ns]
 1   ForecastMonth    269 non-null    float64       
 2   World_BS_Wheat   272 non-null    float64       
 3   World_P_Wheat    272 non-null    float64       
 4   World_D_Wheat    272 non-null    float64       
 5   World_E_Wheat    272 non-null    float64       
 6   Year_Month       272 non-null    period[M]     
 7   World_BS_Coarse  272 non-null    float64       
 8   World_P_Coarse   272 non-null    float64       
 9   World_D_Coarse   272 non-null    float64       
 10  World_E_Coarse   272 non-null    float64       
 11  World_BS_Corn    272 non-null    float64       
 12  World_P_Corn     272 non-null    float64       
 13  World_D_Corn     272 non-null    float64       
 14  World_E_Corn     272 non-null    float64  